In [2]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [9]:
import os

dataset_path = "dataset"

for categoria in ["piezas_buenas", "piezas_malas"]:
    ruta = os.path.join(dataset_path, categoria)
    archivos = os.listdir(ruta)
    print(f"{categoria}: {len(archivos)} imágenes")


piezas_buenas: 245 imágenes
piezas_malas: 90 imágenes


In [3]:
import os
import cv2
import numpy as np

# Ruta a la carpeta dataset
DATASET_PATH = "dataset"
IMG_SIZE = 128

# Etiquetas: piezas buenas = 0, piezas malas = 1
CATEGORIES = ["piezas_buenas", "piezas_malas"]

data = []

# Recorremos ambas categorías
for label, category in enumerate(CATEGORIES):
    folder_path = os.path.join(DATASET_PATH, category)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            # Leer la imagen en escala de grises(hemos cambiado a color porque tenemos fallas por quemaduras)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
             # 🎨 Convertir de BGR a RGB (OpenCV usa BGR por defecto)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Redimensionar a 128x128
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            # Normalizar valores (de 0-255 a 0-1)
            img = img / 255.0
            # Añadir a la lista de datos junto con su etiqueta (0 o 1)
            data.append([img, label])
        except Exception as e:
            print(f"⚠️ Error al procesar {img_path}: {e}")

# Mezclar los datos aleatoriamente
import random
random.shuffle(data)

# Separar datos y etiquetas
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

# Convertir a arrays de numpy
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

print("✅ Imágenes preprocesadas correctamente")
print("🔢 Tamaño de X:", X.shape)
print("🔢 Tamaño de y:", y.shape)


✅ Imágenes preprocesadas correctamente
🔢 Tamaño de X: (335, 128, 128, 3)
🔢 Tamaño de y: (335,)


In [5]:
!pip install scikit-learn


You should consider upgrading via the 'C:\Users\bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 1️⃣ Dividir los datos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Datos divididos:")
print("📊 X_train:", X_train.shape)
print("📊 X_val:", X_val.shape)

# 2️⃣ Crear el modelo CNN


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(X.shape[1], X.shape[2], 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.4),  # Subimos un poco para evitar sobreajuste
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Clasificación binaria
])


# 3️⃣ Compilar
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 4️⃣ Entrenar

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
)

datagen.fit(X_train)
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=30,
    validation_data=(X_val, y_val)
)

# 5️⃣ Graficar resultados
#perdida
plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida Validación')
plt.title("Pérdida durante entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.legend()
plt.show()

#precision
#Muestra qué porcentaje de imágenes clasificó correctamente.
#Si la precisión se mantiene alta en validación → el modelo generaliza bien.

plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión Validación')
plt.title("Precisión durante entrenamiento")
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.legend()
plt.show()


NameError: name 'train_test_split' is not defined

In [40]:
model.save("mi_modelo.keras")
